In [ ]:
from numpy import sqrt, log10, exp
from numpy import log as ln  # "log" is ambiguous
import numpy as np

np.seterr(all='ignore')

dBV_REF = 1  # 1 V
dBu_REF = sqrt(600*1/1000)  # 1 mW into a 600 Ω load = 0.774 V
dBμV_REF = 1e-6  # 1 μV

dBW_REF = 1     # 1 W
dBm_REF = 1e-3  # 1 mW
dBk_REF = 1e3   # 1 kW
dBf_REF = 1e-15 # 1 fW

V_units = ['Vrms', 'Vpk', 'Vpp', 'dBu', 'dBV', 'dBμV']
W_units = ['W', 'dBW', 'dBm', 'dBf', 'dBk']
dml_units = ['V/V', 'W/W', 'dB', 'Np']

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label

# Create an input/output widget for each unit
w = {unit: widgets.FloatText(description=unit + ':') for unit in V_units + W_units + dml_units}
w['Zload'] = widgets.FloatText(description='Zload:', value=50)

HBox([VBox([Label('Voltage units'), *(w[u] for u in V_units)]),
      VBox([Label('Power units'), w['Zload'], *(w[u] for u in W_units)]),
      VBox([Label('Dimensionless units'), *(w[u] for u in dml_units)])
     ])

In [ ]:
# Each node only needs to be connected to one other node.
# These are the simplest relationships to represent:

conversions = {
    ('Vpk',  'Vpp'):  (lambda x: x*2,                  lambda x: x/2),
    ('Vpk',  'Vrms'): (lambda x: x/sqrt(2),            lambda x: x*sqrt(2)),
    ('Vrms', 'dBV'):  (lambda x: 20*log10(x/dBV_REF),  lambda x: 10**(x/20) * dBV_REF),
    ('Vrms', 'dBu'):  (lambda x: 20*log10(x/dBu_REF),  lambda x: 10**(x/20) * dBu_REF),
    ('Vrms', 'dBμV'): (lambda x: 20*log10(x/dBμV_REF), lambda x: 10**(x/20) * dBμV_REF),

    ('W', 'dBW'):  (lambda x: 10*log10(x/dBW_REF),  lambda x: 10**(x/10) * dBW_REF),
    ('W', 'dBm'):  (lambda x: 10*log10(x/dBm_REF),  lambda x: 10**(x/10) * dBm_REF),
    ('W', 'dBk'):  (lambda x: 10*log10(x/dBk_REF),  lambda x: 10**(x/10) * dBk_REF),
    ('W', 'dBf'):  (lambda x: 10*log10(x/dBf_REF),  lambda x: 10**(x/10) * dBf_REF),

    ('Vrms', 'W'):  (lambda V: V**2/w['Zload'].value,  lambda P: sqrt(P*w['Zload'].value)),

    ('V/V', 'W/W'): (lambda x: x**2,         lambda x: sqrt(x)),
    ('V/V', 'Np'):  (lambda x: ln(x),        lambda x: exp(x)),
    ('W/W', 'dB'):  (lambda x: 10*log10(x),  lambda x: 10**(x/10)),
    }

for units, funcs in conversions.items():
    widgets.link((w[units[0]], 'value'), (w[units[1]], 'value'), (funcs[0], funcs[1]))
    
# V and W update each other based on Z, and Z updates W, but W should not update Z
widgets.dlink((w['Zload'], 'value'), (w['W'], 'value'), lambda Z: w['Vrms'].value**2/Z);